# Password cracking

In this day and age, we all use passwords to protect our information and gain access to devices or services. Passwords are very useful, but they also have flaws if not used properly. Over the years, security researchers (and hackers!) have learned that people make really bad and easily guessable passwords, and then reuse them for multiple accounts. This makes life much easier for anyone trying to access your information or steal your stuff!

In this workshop, we're going to have a look at just a few of the ways an adversary might try to break your password. By the end you'll know how to make a strong password, and you'll be able to tell your family and friends how to do it too!

We'll be doing a little bit of programming in Python, but don't worry if you haven't done any before. Most of the code is already written in this notebook. All you have to do is execute it, which you can do by clicking in the cell with the code and pressing Shift+Enter. **Have a go with the cell below!** 

(Activities for you to try will be in bold throughout this notebook).


In [1]:
# The hash at the beginning of this line means it won't be executed. It's called a "comment", and is used to explain code.
print('Congratulations, you have executed a print statement!')

Congratulations, you have executed a print statement!


Also execute the cell below- it imports functions from a file called "passwordcracking" that we're going to need.

In [3]:
from passwordcracking import *

## Cryptographic hash functions

Have you ever wondered what happens to your password once you enter it into the program, website or whatever you're trying to access? Obviously, the system wants to somehow check you've entered the correct one. Perhaps it could check it against a stored copy of the correct password. But then your password would need to be saved somewhere, just waiting for someone to find and read it! Worse, if you were logging into something online, this would mean sending your password over the internet making it very easy for an adversary to spot it.

A cryptographic hash (or just hash) is a type of function designed to solve this problem. A hash function takes your password and returns a unique, random-looking string. There are many hash functions- one common one is called MD5 (which stands for Message Digest 5). The MD5 hash of the word "password" is 

> MD5(password)=5f4dcc3b5aa765d61d8327deb882cf99

You can calculate the hash of the word "password" in Python using one of the functions imported from the file above.

In [5]:
MD5hash("passwore")

'a826176c6495c5116189db91770e20ce'

Hash functions are designed so that if you make any tiny change to the input, the output changes drastically. 

**Try changing a letter in the red word "password" in the code above and see what happens.**

A good cryptographic hash function has three important properties:
   1. Given a hash, it is extremely difficult to find an input that results in that hash.
   2. Given an input, it is extremely difficult to find a second input such that the hashes are the same.
   3. It is extremely difficult to find any two inputs that result in the same hash.
   
The first property is the most important for password uses- it means the hash function is irreversible. (The other two properties are very important for other hash uses, but we won't be looking into them here).

So now we have a solution for storing and sending your password in so-called plaintext. Instead, when you set your password, a hash of it is stored instead of the password itself. When you're next trying to log in to your service, the app will calculate your hash, and compare that to the stored one. Great! We're done!

... Not quite. Unfortunately, even though the hash is irreversible, that doesn't mean it can't be attacked. Often, an adversary can see the hash, and can use a range of techniques to try to work out the password that created it. We'll now look at some of those methods.

## Brute force

The most obvious way to attack a hash is to cycle through all possible passwords, calculating the hashes until you get a match. This is called a Brute Force attack. 

We can calculate how many you'd have to check. Suppose your password was actually a four-digit number you used for a combination lock on your bike. There are 10 possibilities for each position, so the total number of options is 
$$ 10*10*10*10 = 10^{4}=10000, $$

specifically, the numbers 0000-9999. This wouldn't take a computer long at all- in fact, a dedicated thief trying a combination each second could steal your bike within a few hours. 

We can do this calculation in Python. The arithmetic symbols are:

   * Addition and subtraction are exactly as expected e.g. `1+1`, `5-3`
   * Multiplication uses an asterix e.g. `4*5`
   * Division uses a forward slash e.g. `20/4`
   * Exponentiation uses double asterix e.g. `2**3`
   * Brackets work as expected e.g. `5*(1+3)`

In [ ]:
# Code to calculate the bike lock problem. 
Combinations=10**4
CombsPerSecond=1
SecsPerMin=60
MinPerHr=60

NumHrs= Combinations/(CombsPerSecond*SecsPerMin*MinPerHr)

print('The number of combinations is', Combinations) 
print('At one combination each second, it would take',NumHrs, 'hours to break')  

Fortunately, for our passwords we can use many more characters and make it much longer. 

Suppose each character of your password could be any letter (uppercase or lowercase), number, or special character that appears on the number keys. That's 72 options for each character. Suppose also that your password was 11 letters long. **In the cell below, calculate how many different passwords there could be. Show that if a computer could calculate a billion ($10^9$) hashes a second, it would take more than 8000 years to break.** That would test the patience of any hacker!

In [11]:
# Fill in the blanks below to solve the 11 character password problem.
Combinations=72**11
CombsPerSecond=1e9
SecsPerMin=60
MinsPerHr=60
HrsPerDay=24
DaysPerYear=365
CombsPerYear=CombsPerSecond*SecsPerMin*MinsPerHr*HrsPerDay*DaysPerYear

NumYrs=Combinations/CombsPerYear

print('The number of combinations is', Combinations) 
print('At one combination each second, it would take', NumYrs, 'years to break')

The number of combinations is 269561249468963094528
At one combination each second, it would take 8547.731147544491 years to break


## Mask attack

A mask attack is a step up from a brute force attack. The "mask" allows you to set conditions on the characters you'll test in each of the password character positions. This is useful, because we know about humans and how they design passwords.

A very common pattern that people use to create their password is *NameYYYY* where YYYY represents their birth year. We can use a mask to tell the computer to check for upper case letters in the first position, then three lower case letters, then four numbers.

We're going to try this using a tool called *hashcat*- a professional password recovery tool used by security experts worldwide. To use hashcat, you need to enter a series of mysterious looking commands, which I'll explain as we go along.

To create the mask command in hashcat, you create a string which tells the program what character sets to look for in each position. The built in options are:
   * `?l = abcdefghijklmnopqrstuvwxyz`
   * `?u = ABCDEFGHIJKLMNOPQRSTUVWXYZ`
   * `?d = 0123456789`
   * `?h = 0123456789abcdef`
   * `?H = 0123456789ABCDEF`
   * `?s = «space»!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
   * `?a = ?l?u?d?s`
   * `?b = 0x00 - 0xff`

So for example, if you knew you were looking for a PIN of length 4, the command you would enter is `?d?d?d?d`. If you knew you were looking for a five letter name starting with a capital, then a year, you'd enter `?u?l?l?l?d?d?d?d`.

Let's see this in action.

In [13]:
# recovering a PIN 
HASH="e82c4b19b8151ddc25d4d93baf7b908f"
MASK="?d?d?d?d"

MaskAttack(HASH,MASK)

CalledProcessError: Command '['hashcat', '-a', '3', '-m', '0', 'e82c4b19b8151ddc25d4d93baf7b908f', '--show']' returned non-zero exit status 255.

In [ ]:
# recovering a five letter name and date (NameYYYY)

HASH="42f31227dbe851ff9a1a74f254d4a5f3"
MASK="?u?l?l?l?l?d?d?d?d"

MaskAttack(HASH,MASK)

Hashcat also allows you to create a custom character set. For example, if you know the password is entirely made up of lowercase OR digits, you can define a character set called 1 (you can have up to 4) by adding in the command `?l?d`, and then call it using `?1` just the same as the default character sets.

In [ ]:
# recover a 4 character password where each character is either a lowercase letter or a digit 
HASH='f2e49af795161e14acf9d9245473a368'
CUSTOMSET='?l?d' 
MASK='?1?1?1?1'

MaskAttack(HASH,MASK,CUSTOMSET)

**Now it's your turn. Try to break the following MD5 hashes using a mask attack.** You can copy the hashcat commands from above, and then insert the new hash and write your own mask and/ or custom character set.
   * `2d3c7cd7e9224d7100570660d4530455` given it's a 6 digit PIN
   * `0b01256584505412f939a973304fdd87` given it's a four letter name and birth year 
   * `a1e22f0fbe6c32997616af6018f8dcd3` given it's five letter name and birth year, but the user likes to sometimes replace letters of their name with special characters.

In [ ]:
# Exercise
# write a mask for this hash, given it's a 6 digit PIN
HASH= '2d3c7cd7e9224d7100570660d4530455'
MASK=

MaskAttack(HASH,MASK)

In [ ]:
# Exercise
# write a mask for this hash, given it's a 4 letter name and a birth year
HASH='0b01256584505412f939a973304fdd87'
MASK=

MaskAttack(HASH,MASK)

In [ ]:
#Exercise
#write a mask for this hash given it's a 5 letter name and birth year, and the user likes to replace letters with special chars
HASH='a1e22f0fbe6c32997616af6018f8dcd3'
CUSTOMSET=
MASK=

MaskAttack(HASH,MASK,CUSTOMSET)

There are many more options that can be checked using a mask attack, but for now we're going to move on.

## Dictionary attacks

You probably noticed that the mask attacks in the last exercise can take a while, even though we had a lot of hints for the form of the password. Another type of attack is a *Dictionary attack*. This type of attack exploits the fact that we know humans tend to be creatures of habit, and they like to use passwords that are easy to remember, for example, English words or their favourite person's name.

Wordlists are available all over the internet, although master hashcrackers build their own and guard them jealously. We're using one that contains about 10,000 of the top passwords used in English speaking countries. Here's a look at the first 100 of those passwords.

In [4]:
top10000="%homepath%/Documents/Password_cracking_workshop/top10000"

with open("./top10000") as pwdfile:
    for i in range(100):
        print(pwdfile.readline())

123456

password

12345678

qwerty

123456789

12345

1234

111111

1234567

dragon

123123

baseball

abc123

football

monkey

letmein

696969

shadow

master

666666

qwertyuiop

123321

mustang

1234567890

michael

654321

superman

1qaz2wsx

7777777

121212

000000

qazwsx

123qwe

killer

trustno1

jordan

jennifer

zxcvbnm

asdfgh

hunter

buster

soccer

harley

batman

andrew

tigger

sunshine

iloveyou

2000

charlie

robert

thomas

hockey

ranger

daniel

starwars

klaster

112233

george

computer

michelle

jessica

pepper

1111

zxcvbn

555555

11111111

131313

freedom

777777

pass

maggie

159753

aaaaaa

ginger

princess

joshua

cheese

amanda

summer

love

ashley

6969

nicole

chelsea

biteme

matthew

access

yankees

987654321

dallas

austin

thunder

taylor

matrix

william

corvette

hello

martin

heather



We can perform a dictionary attack using the function `DictAttack` as below.

In [9]:
HASH= '21232f297a57a5a743894a0e4a801fc3'
DICTFILE='./top10000'

DictAttack(HASH,DICTFILE)

The password for hash  21232f297a57a5a743894a0e4a801fc3  is  admin


['21232f297a57a5a743894a0e4a801fc3', 'admin']

### Dictionary attack with rules

Often, people create more complex passwords by starting with a simple password and modifying it.  For example, some use leetspeak, swapping letters for similar looking numbers and special characters. 

   * password -> p@ssw0rd

Another example is swapping the case for one or more characters in the password.

   * password -> PassWord

In hashcat this type of transformation is called a ‘rule’.  Hashcat comes with rule files where the details of the transformation are documented.  Hashcat comes with some standard rule files – a couple of the most straight forward are those for leetspeak transformation (‘leetspeak.rule’) and case toggling, such as we’ve seen in the examples above.

There are actually a number of case toggling rule files, depending on the number of characters being toggled.

For example, ‘toggles1.rule’ case swaps 1 character, while ‘toggles3.rule’ case swaps 3 of the characters.

In [6]:
# Four of the rules files
LEETSPEAK='rules/leetspeak.rule'
TOGGLES1='rules/toggles1.rule'
TOGGLES2='rules/toggles2.rule'
TOGGLES3='rules/toggles3.rule'

Let's try to recover the password for the hash `b4247afabe9109b8cb9f04a93fdc565a` using the dictionary file and the leetspeak rule.

In [8]:
HASH= 'b4247afabe9109b8cb9f04a93fdc565a'

DictAttack(HASH,,[LEETSPEAK])

IndexError: list index out of range

You can also combine rule files to create more complicated passwords.  The following password contains both leetspeak and a single case swap.

   * Password -> p@ssWord

Your turn- **recover the password for the hash `42d679d562e0c108ab3c5fbb7bda4064` using the dictionary file and the leetspeak and toggles1 rules.** You'll need to put the rules in a list like `[RULE1, RULE2]`.

In [ ]:
# Exercise
# recover the password for the hash, using the dictionary file and the leetspeak and toggles1 rules
HASH= '42d679d562e0c108ab3c5fbb7bda4064'

DictAttack(HASH,DICTFILE,[])

# Challenge

**Using all the tricks you've learned, recover as many passwords you can of the following list of hashes.** Keep track of the ones you've recovered in the cell below.

   * `93cfed079004d4370010cb17bda1daab`
   * `e7e85de09cfe441003ced1a82bb611a6`
   * `f01e0d7992a3b7748538d02291b0beae`
   * `e31e83bdffa9bfb546175af6d6da3e4b`
   * `9bc3d40134996aba6d821fa60bf1504a`
   * `4a4f687f8864cfb111fb2c7c54bbe27b`
   * `c9754fb19e1371b8cef0924113146f5e`
   * `62c8ad0a15d9d1ca38d5dee762a16e01`
   * `40be4e59b9a2a2b5dffb918c0e86b3d7`
   * `beb8e16d1f4dcdf54297880e2bedddc9`
   * `09117a8f8691865023cb388284a1a0e9`
   * `e91a81cacf6ba680079a9fbfd32893eb`
   * `b0baee9d279d34fa1dfd71aadb908c3f`
   * `531537102901716809ed185e76290202`
   * `7a75a532aaab234ad4bd33ed67e67242`
   * `1e0bb5b62610a9e76c3053bc2aceafd2`
   * `d39742395ddcc548c21ee1dadd0130f0`
   * `b7e52ac21a810fe8132996c5226b62fa`
   * `4121b8c8da6e0dcf056ffe9fc15b74b9`
   
These passwords are known to be one of:

    1)      Commonly used password (try your dictionary file!)

    2)      Common password with a leetspeak transformation

    3)      Common password with two characters case swapped (toggled)

    4)      Passwords consisting of all upper case characters, 5 characters long (e.g. ‘ABCDE’)

    5)      Passwords consisting of all special characters, between 2 and 5 characters long (e.g. ‘%#@’)

    6)      Common passwords with 1 character toggled and a leetspeak transformation

In [ ]:
# Enter your code here


In [ ]:
# print your solved hashes 
PrintSolved()

## Conclusion

You have now tried just a few of the ways a hacker might try to break your password. Through these activities, you've learned that when it comes to passwords
   * longer is better
   * predictable patterns are easy to guess
   * single words are easy to guess too, even if you use funky characters.

Some hackers are very clever, and use even more sophisticated techniques than we've seen here. So how do you make a secure password? 

ASD, through the Australian Cyber Security Centre, provides password advice on the website 

https://www.staysmartonline.gov.au/

The advice includes
  * Think of a passphrase that is made up of at least four words, including at least 13 characters, for example 'horsecupstarshoe'. Make it meaningful to you so it is easy to remember.
  * Install a password manager on your computer, smartphone or tablet. It will generate and remember secure passwords for you and some password managers will sync across your devices.
  * Use two-factor authentication.
  * Don't use the same password for multiple services or websites.
  * Use password tiers- unique and complex passwords for high risk accounts such as online shopping or social media, and less complex ones for low risk ones such as newsletters.
  * Never give your password to anyone, ever!
  
For more tips for how to create strong passwords and stay safe online, check out the website.
  
  

In [ ]:
EmptyPot()